In [24]:
%%writefile Paris_Airbnb.py

import streamlit as st
import plotly.express as px
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, mean_squared_error, r2_score
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from io import BytesIO
from streamlit_option_menu import option_menu  # Importamos la librería

# Definimos la instancia de streamlit
@st.cache_data
def load_data():
    df = pd.read_csv("Base_Paris_Final.csv")
    
    # Definir las columnas categóricas
    categorical_columns = ['host_is_superhost', 'host_response_time', 'host_identity_verified', 'property_type', 'room_type']

    # Crear un diccionario para almacenar las tablas de frecuencia
    frequency_tables = {}

    # Generar tablas de frecuencia para cada columna categórica
    for column in categorical_columns:
        frequency_table = df.groupby('host_id')[column].value_counts().unstack(fill_value=0)
        frequency_tables[column] = frequency_table

    # Eliminar los valores "No captured" de todas las tablas de frecuencia
    for column, table in frequency_tables.items():
        frequency_tables[column] = table.drop('No captured', axis=1, errors='ignore')

    return df, frequency_tables

# Cargar los datos
df, frequency_tables = load_data()

# Función para crear el PDF con una plantilla y agregar los datos
def create_pdf_with_template(host_id, total_reviews, avg_rating, avg_cleanliness, avg_location, avg_value):
    # Cargar la plantilla PDF
    template_path = "template.pdf"
    template_pdf = PdfReader(template_path)
    output_pdf = PdfWriter()

    # Crear un buffer para almacenar el nuevo PDF
    buffer = BytesIO()

    # Crear el lienzo para añadir texto y gráficos
    pdf_canvas = canvas.Canvas(buffer, pagesize=letter)
    
    # Ajustar el color de la fuente a rojo
    pdf_canvas.setFillColorRGB(1, 0, 0)  # Color rojo (RGB: 1, 0, 0)
    
    # Calcular el tamaño proporcional de la fuente en función del tamaño del PDF
    # Aquí usamos el tamaño de la página estándar (letter) que es 612 x 792 puntos
    font_size = 12  # Tamaño de fuente base
    pdf_width, pdf_height = letter
    proportional_font_size = font_size * (pdf_width / 612)  # Escalar en función del ancho del PDF

    # Establecer la fuente con el tamaño proporcional
    pdf_canvas.setFont("Helvetica", proportional_font_size)

    # Añadir el contenido a la plantilla existente
    pdf_canvas.drawString(100, 750, f"Host ID: {host_id}")
    pdf_canvas.drawString(100, 730, f"Total de Reseñas: {total_reviews}")
    pdf_canvas.drawString(100, 710, f"Promedio de Calificación: {avg_rating:.2f}")
    pdf_canvas.drawString(100, 690, f"Promedio de Limpieza: {avg_cleanliness:.2f}")
    pdf_canvas.drawString(100, 670, f"Promedio de Ubicación: {avg_location:.2f}")
    pdf_canvas.drawString(100, 650, f"Promedio de Valor: {avg_value:.2f}")

    pdf_canvas.showPage()
    pdf_canvas.save()

    # Fusionar la plantilla original con el contenido generado
    buffer.seek(0)
    new_pdf = PdfReader(buffer)
    
    # Añadir cada página del nuevo contenido a la plantilla
    for page_num in range(len(template_pdf.pages)):
        output_pdf.add_page(template_pdf.pages[page_num])
    for page_num in range(len(new_pdf.pages)):
        output_pdf.add_page(new_pdf.pages[page_num])

    # Guardar el archivo final
    output_buffer = BytesIO()
    output_pdf.write(output_buffer)
    output_buffer.seek(0)

    return output_buffer

# 1. CREACIÓN DE LA SIDEBAR
st.sidebar.markdown(
    """
    <style>
    [data-testid="stSidebar"] {
        background-color: #FF0000;
    }
    [data-testid="stSidebar"] h1, [data-testid="stSidebar"] h2, [data-testid="stSidebar"] h3 {
        color: #FFFFFF;
    }
    </style>
    """,
    unsafe_allow_html=True
)

# Agregar imagen al sidebar (Asegúrate de que la ruta sea correcta)
st.sidebar.image("222.png", use_column_width=True)

st.sidebar.title("DASHBOARD")
st.sidebar.header("Sidebar")
st.sidebar.subheader("________________________________")

import base64

# 2. CREACIÓN DEL MENÚ CON STREAMLIT OPTION MENU
with st.sidebar:
    Frames = option_menu(
        menu_title="Menú Principal",
        options=["Datos generales", "Ubicación y métricas", "Precio Ideal", "Recomendaciones"],
        icons=["clipboard-data", "geo-alt", "cash-stack", "lightbulb"],
        menu_icon="menu-button-wide",
        default_index=0,
    )

if Frames == "Datos generales":
    # Cargar la imagen (Asegúrate de que la ruta sea correcta)
    image_path = "General.png"
    with open(image_path, "rb") as image_file:
        image_base64 = base64.b64encode(image_file.read()).decode("utf-8")
    
    st.markdown(
        f"""
        <h1 style="display: flex; align-items: center;">
            <img src="data:image/png;base64,{image_base64}" width="64" style="margin-right: 10px;"/>
            <span>Datos generales</span>
        </h1>
        """,
        unsafe_allow_html=True
    )
    
    host_id = st.sidebar.selectbox("Selecciona el host_id:", df['host_id'].unique())
    filtered_df = df[df['host_id'] == host_id]

    st.markdown("### Tablas de Frecuencia para el ID del cliente seleccionado")
    for column, table in frequency_tables.items():
        st.markdown(f"#### Frecuencia para {column}:")
        st.dataframe(table.loc[host_id])

        freq_data = table.loc[host_id].reset_index()
        freq_data.columns = [column, 'count']
        fig = px.bar(freq_data, x=column, y='count', title=f"Frecuencia de {column} para el Host ID {host_id}", color='count', color_continuous_scale='Reds')
        st.plotly_chart(fig, use_container_width=True)

elif Frames == "Ubicación y métricas":
    host_accommodations = df.groupby('host_id').size().reset_index(name='accommodation_count')
    top_15_hosts = host_accommodations.nlargest(15, 'accommodation_count')['host_id']
    top_host_id = st.sidebar.selectbox("Selecciona uno de los 15 host_id con más alojamientos:", top_15_hosts)
    filtered_top_df = df[df['host_id'] == top_host_id]

    st.markdown("### Mapa de Alojamientos en París para el Top Host ID")
    fig_top_map = px.scatter_mapbox(
        filtered_top_df,
        lat="latitude",
        lon="longitude",
        hover_name="name",
        hover_data=["host_id", "room_type", "price"],
        color_discrete_sequence=["blue"],
        zoom=10,
        height=600
    )
    fig_top_map.update_layout(mapbox_style="open-street-map")
    st.plotly_chart(fig_top_map, use_container_width=True)

    st.markdown("### Explicación de los 15 Host ID con Más Alojamientos")
    st.write("Los 15 host_id con más alojamientos representan a los anfitriones que tienen la mayor cantidad de propiedades listadas en Airbnb en París.")

    host_id = st.sidebar.selectbox("Selecciona el host_id:", df['host_id'].unique())
    filtered_df = df[df['host_id'] == host_id]

    st.markdown("### Mapa de Alojamientos en París")
    fig_map = px.scatter_mapbox(
        filtered_df,
        lat="latitude",
        lon="longitude",
        hover_name="name",
        hover_data=["host_id", "room_type", "price"],
        color_discrete_sequence=["red"],
        zoom=10,
        height=600
    )
    fig_map.update_layout(mapbox_style="open-street-map")
    st.plotly_chart(fig_map, use_container_width=True)

    st.markdown("### Métricas Agregadas para el Host ID Seleccionado")
    total_reviews = filtered_df['number_of_reviews'].sum()
    avg_rating = filtered_df['review_scores_rating'].mean()
    avg_cleanliness = filtered_df['review_scores_cleanliness'].mean()
    avg_location = filtered_df['review_scores_location'].mean()
    avg_value = filtered_df['review_scores_value'].mean()

    st.metric(label="Total de Reseñas", value=total_reviews)
    st.metric(label="Promedio de Calificación", value=round(avg_rating, 2))
    st.metric(label="Promedio de Limpieza", value=round(avg_cleanliness, 2))
    st.metric(label="Promedio de Ubicación", value=round(avg_location, 2))
    st.metric(label="Promedio de Valor", value=round(avg_value, 2))

    # Botón para generar el PDF
    if st.button("Descargar PDF"):
        pdf_buffer = create_pdf_with_template(host_id, total_reviews, avg_rating, avg_cleanliness, avg_location, avg_value)
        st.download_button(label="Descargar PDF", data=pdf_buffer, file_name="reporte_host.pdf", mime="application/pdf")
        
elif Frames == "Precio Ideal":
    st.markdown("## Predicción del Precio de un Nuevo Alojamiento")

    # Seleccionar las columnas relevantes para la regresión
    columns = ['price', 'bedrooms', 'accommodates', 'beds', 'review_scores_location', 'number_of_reviews', 'availability_365', 'minimum_nights', 'reviews_per_month']
    df_regression = df[columns].dropna()

    # Definir variables dependientes e independientes
    X = df_regression[['bedrooms', 'accommodates', 'beds', 'review_scores_location', 'number_of_reviews', 'availability_365', 'minimum_nights', 'reviews_per_month']]
    y = df_regression['price']

    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Crear el modelo de regresión lineal
    model = LinearRegression()

    # Entrenar el modelo
    model.fit(X_train, y_train)

    # Realizar predicciones en el conjunto de prueba
    y_pred = model.predict(X_test)

    # Calcular métricas de rendimiento
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    accuracy_percentage = r2 * 100

    # Solicitar al usuario que ingrese los datos
    bedrooms = st.number_input("Ingrese el número de habitaciones:", min_value=0.0, step=1.0)
    accommodates = st.number_input("Ingrese el número de personas que puede alojar:", min_value=0.0, step=1.0)
    beds = st.number_input("Ingrese el número de camas:", min_value=0.0, step=1.0)
    review_scores_location = st.number_input("Ingrese la puntuación de la ubicación (1-5):", min_value=1.0, max_value=5.0, step=0.1)
    number_of_reviews = st.number_input("Ingrese el número de reseñas:", min_value=0.0, step=1.0)
    availability_365 = st.number_input("Ingrese la disponibilidad en días por año:", min_value=0.0, step=1.0)
    minimum_nights = st.number_input("Ingrese el número mínimo de noches:", min_value=0.0, step=1.0)
    reviews_per_month = st.number_input("Ingrese el número de reseñas por mes:", min_value=0.0, step=0.1)

    if st.button("Predecir Precio"):
        # Definir un diccionario para almacenar los datos de entrada
        new_data = {
            'bedrooms': bedrooms,
            'accommodates': accommodates,
            'beds': beds,
            'review_scores_location': review_scores_location,
            'number_of_reviews': number_of_reviews,
            'availability_365': availability_365,
            'minimum_nights': minimum_nights,
            'reviews_per_month': reviews_per_month
        }

        # Convertir los datos de entrada en un DataFrame
        new_df = pd.DataFrame([new_data])

        # Realizar la predicción utilizando el modelo de regresión lineal
        predicted_price = model.predict(new_df)

        st.write(f"El precio ideal para el nuevo alojamiento es: {predicted_price[0]:.2f}")
        st.write(f"Ten en cuenta que este es solo un estimado, hecho por un modelo predictivo de regresión lineal múltiple, con {accuracy_percentage:.2f}% de efectividad y que el precio real puede variar.")
        st.write("¡Gracias por utilizar nuestro servicio!")

    # Mostrar las métricas de rendimiento al final
    st.write(f"Mean Squared Error (MSE): {mse}")
    st.write(f"R² Score: {r2}")
    st.write(f"El modelo tiene una precisión del {accuracy_percentage:.2f}%")

elif Frames == "Recomendaciones":
    st.markdown("## Evaluación de Precios de Alojamientos por Host ID")

    # Seleccionar el host_id
    host_id = st.sidebar.selectbox("Selecciona el host_id:", df['host_id'].unique())

    # Filtrar los datos basados en el host_id seleccionado
    filtered_df = df[df['host_id'] == host_id]

    # Mostrar el número total de alojamientos
    total_accommodations = filtered_df.shape[0]
    st.write(f"Total de alojamientos: {total_accommodations}")

    # Obtenemos el límite superior y el límite inferior de la columna objetivo
    Max = df['price'].max()
    Min = df['price'].min()
    Limites = [Min, Max]

    # Categorización de variables
    # Declaramos 2 intervalos 
    intervalos = np.linspace(9, 486, 3)

    # Creamos las categorías 
    categorias = ["Min.Price", "Max.Price"]

    # Finalmente creamos las categorías en la columna numérica
    df['price'] = pd.cut(x=df['price'], bins=intervalos, labels=categorias)
    df = df.dropna(subset=['price'])

    # Declaramos las variables dependientes e independientes para la regresión Logística
    Vars_Indep = df[['bathrooms', 'bedrooms', 'beds']]
    Var_Dep = df['price']

    # Redefinimos las variables 
    X = Vars_Indep
    y = Var_Dep

    # Eliminar filas con valores NaN
    X = X.dropna()
    y = y.dropna()

    # Asegurarse de que X e y tengan el mismo número de muestras
    X, y = X.align(y, join='inner', axis=0)

    # Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=None)

    # Se escalan todos los datos
    escalar = StandardScaler()

    # Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos fit_transform
    X_train = escalar.fit_transform(X_train)
    X_test = escalar.transform(X_test)

    # Definimos el algoritmo a utilizar
    algoritmo = LogisticRegression()

    # Entrenamos el modelo
    algoritmo.fit(X_train, y_train)

    # Realizamos una predicción
    y_pred = algoritmo.predict(X_test)

    # Verifico la matriz de Confusión
    matriz = confusion_matrix(y_test, y_pred)
    st.write('Matriz de Confusión')
    st.write(matriz)

    # Calculo la precisión del modelo
    precision = precision_score(y_test, y_pred, average="binary", pos_label="Min.Price")
    st.write('Precisión del modelo:')
    st.write(precision)

    # Calculo la exactitud del modelo
    exactitud = accuracy_score(y_test, y_pred)
    st.write('Exactitud del modelo:')
    st.write(exactitud)

    # Calculo la sensibilidad del modelo
    sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="Min.Price")
    st.write('Sensibilidad del modelo:')
    st.write(sensibilidad)

    st.markdown("### Evaluación de Precios de Alojamientos")

    # Crear un DataFrame para almacenar las evaluaciones
    evaluation_df = pd.DataFrame(columns=['Alojamiento ID', 'Precio Actual', 'Recomendación'])

    for index, row in filtered_df.iterrows():
        bathrooms = row['bathrooms']
        bedrooms = row['bedrooms']
        beds = row['beds']
        current_price = row['price']

        # Crear un DataFrame con los datos del alojamiento
        new_data = pd.DataFrame([{
            'bathrooms': bathrooms,
            'bedrooms': bedrooms,
            'beds': beds
        }])

        # Escalar los datos de entrada
        new_data_scaled = escalar.transform(new_data)

        # Realizar la predicción utilizando el modelo de LogisticRegression
        predicted_category = algoritmo.predict(new_data_scaled)

        # Determinar la recomendación basada en el precio actual y la categoría predicha
        if predicted_category[0] == "Max.Price":
            if current_price >= Max:
                recommendation = "El precio está adecuado."
            else:
                recommendation = "Deberías subir el precio."
        elif predicted_category[0] == "Min.Price":
            if current_price <= Min:
                recommendation = "El precio está adecuado."
            else:
                recommendation = "Deberías bajar el precio."

        # Agregar la evaluación al DataFrame
        new_row = pd.DataFrame({
            'Alojamiento ID': [index],
            'Precio Actual': [current_price],
            'Recomendación': [recommendation]
        })
        evaluation_df = pd.concat([evaluation_df, new_row], ignore_index=True)

    # Mostrar el DataFrame de evaluaciones
    st.dataframe(evaluation_df)


Overwriting Paris_Airbnb.py


460047164
streamlit run Paris_Airbnb.py